# Algoritmo genético simple

Los Algoritmos Genéticos (AGs) son métodos adaptativos que pueden usarse para resolver problemas de búsqueda y optimización. Están basados en el proceso genético de los organismos vivos. A lo largo de las generaciones, las poblaciones evolucionan en la naturaleza de acorde con los principios de la selección natural y la supervivencia de los más fuertes, postulados por Darwin (1859). Por imitación de este proceso, los Algoritmos Genéticos son capaces de ir creando soluciones para problemas del mundo real. La evolución de dichas soluciones hacia valores óptimos del problema depende en buena medida de una adecuada codificación de las mismas. 

<img src= "https://www.miguelvedoya.com/wp-content/uploads/2017/06/Proceso.png">

Tomado de http://www.sc.ehu.es/ccwbayes/docencia/mmcc/docs/temageneticos.pdf

Los algoritmos geneticos simples constan de cuatro pasos: 

1. Población inicial
2. Selección de padres
3. Recombinación o crossover
4. Mutación

Según la codificación y el problema que se este solucionando se debe agregar un paso de **reparación**. Este paso es necesario para recuperar la factibilidad. 

<img src="https://i.ibb.co/HFyfmV6/Algoritmo-genetico.png" alt="Algoritmo-genetico" border="0">

Tomado de: **Glover, F. W., & Kochenberger, G. A. (Eds.). (2006). Handbook of metaheuristics (Vol. 57). Springer Science & Business Media.**

# Problema de la mochila (Knapsack problem)

<img src= "https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Knapsack.svg/220px-Knapsack.svg.png">

# Parámetros de prueba

A continuación, se presentan los parámetros de prueba de nuestro problema de la mochila 0-1

In [1]:
import numpy as np #Vamos a utilizar la libreria numpy
import operator #Paquete para ordenar un diccionario
import time #Paquete para medir el tiempo computacional

np.random.seed(18) 

n = 20 #Número de ítems a empacar
v_i = {i:np.random.randint(100, 200) for i in range(n)} #beneficio del ítem i (valores entre 100 y 200)
w_i = {i:np.random.randint(50, 150) for i in range(n)} #Peso del ítem i (valores entre 50 y 100)
W = round(sum(w_i.values())*0.6) #Peso máximo de la mochila
print(f"Los beneficios de nuestros ítems son {v_i}")
print(f"Los pesos de nuestros ítems son {w_i}")
print(f"La capacidad de la mochila es {W}")

Los beneficios de nuestros ítems son {0: 142, 1: 119, 2: 169, 3: 162, 4: 149, 5: 146, 6: 166, 7: 108, 8: 124, 9: 198, 10: 117, 11: 105, 12: 185, 13: 162, 14: 110, 15: 175, 16: 155, 17: 136, 18: 168, 19: 165}
Los pesos de nuestros ítems son {0: 117, 1: 123, 2: 55, 3: 64, 4: 120, 5: 135, 6: 63, 7: 58, 8: 68, 9: 114, 10: 143, 11: 59, 12: 127, 13: 60, 14: 108, 15: 127, 16: 65, 17: 109, 18: 53, 19: 100}
La capacidad de la mochila es 1121


In [2]:
def actualizo_incumbente(FO_Incumbente, Incumbente, Peso_Incumbente, FO, solucion, Peso_sol):
    
    if FO > FO_Incumbente:
        Incumbente = solucion.copy()
        FO_Incumbente = FO
        Peso_Incumbente = Peso_sol
        
    return FO_Incumbente, Incumbente, Peso_Incumbente

# Codificación

El problema de la mochila 0-1 puede ser codificado con un vector de tamaño $n$. La posición del vector corresponde al tipo de ítem y el contenido de la posición (0 o 1) indica si ese ítem se agrega o no a la mochila.

<img src="https://i.ibb.co/02bjnbB/Codificaci-n-Mochila.png" alt="Codificaci-n-Mochila" border="0">

# 1. Población inicial

El primer paso del algoritmo generico simple consiste en generar la población inicial de individuos. El tamaño de la población estará definido por el parametro $Size$. Este parametro indica el numero de individuos que se van a crear de forma aleatoria. 

El siguiente código crea una población aleatoria de tamaño $Size$ y la respectiva incumbente.

In [3]:
def Crea_Poblacion(Size, n):
    FO_Incumbente = -10 
    Incumbente = []
    Peso_Incumbente = 123
    
    Poblacion = []
    FO_Poblacion = []
    Peso_Poblacion = []
    
    for i in range(Size):
        solucion = [0 for i in range(n)]
        peso_total, FO = 0, 0
        
        for j in range(len(solucion)):
            if np.random.rand() < 0.5 and (peso_total + w_i[j]) <= W:
                solucion[j] = 1
                peso_total+=w_i[j]
                FO+=v_i[j]
                
        FO_Incumbente, Incumbente, Peso_Incumbente = actualizo_incumbente(FO_Incumbente, Incumbente, Peso_Incumbente, FO, solucion, peso_total)
            
        Poblacion.append(solucion)
        FO_Poblacion.append(FO)
        Peso_Poblacion.append(peso_total)
        
    return Poblacion, FO_Poblacion, Peso_Poblacion, Incumbente, FO_Incumbente, Peso_Incumbente

Vamos a crear una población de 10 individuos

In [4]:
Size = 10

Poblacion, FO_Poblacion, Peso_Poblacion, Incumbente, FO_Incumbente, Peso_Incumbente = Crea_Poblacion(Size, n)

In [5]:
print(Poblacion)

[[0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1], [1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1], [0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0]]


In [6]:
print(FO_Poblacion)

[1326, 1404, 1510, 1568, 1134, 1342, 1421, 579, 1610, 1298]


In [8]:
print(Incumbente)
print(FO_Incumbente)
print(Peso_Incumbente)

[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0]
1610
1067


# Selección

Cuando ya se tiene la población contruida se pasa a hacer la selección de los padres. En este caso probaremos dos tipos de selección.

1. Selección por ruleta
2. Selección por torneo

Tomado de: http://sabia.tic.udc.es/mgestal/cv/aaggtutorial/node9.html

# Selección por ruleta 

A cada uno de los individuos de la población se le asigna una parte proporcional a su ajuste de una ruleta, de tal forma que la suma de todos los porcentajes sea la unidad. Los mejores individuos recibirán una porción de la ruleta mayor que la recibida por los peores. Generalmente la población está ordenada en base al ajuste por lo que las porciones más grandes se encuentran al inicio de la ruleta. Para seleccionar un individuo basta con generar un número aleatorio del intervalo [0..1] y devolver el individuo situado en esa posición de la ruleta. Esta posición se suele obtener recorriendo los individuos de la población y acumulando sus proporciones de ruleta hasta que la suma exceda el valor obtenido.

Es un método muy sencillo, pero ineficiente a medida que aumenta el tamaño de la población (su complejidad es $ O(n^2)$). Presenta además el inconveniente de que el peor individuo puede ser seleccionado más de una vez.

En mucha bibliografía se suele referenciar a este método con el nombre de Selección de Montecarlo.

In [9]:
Total = sum(FO_Poblacion)
FO_Poblacion_porc = {i : FO_Poblacion[i]/Total for i in range(len(FO_Poblacion))}
orden = sorted(FO_Poblacion_porc.items(), key=operator.itemgetter(1), reverse=True)

def Seleccion_ruleta(orden):
    index = []
    for i in range(2):  
        aleatorio = np.random.rand()
        ayuda = 0.0
        for i in orden:
            ayuda+=i[1]
            if aleatorio <= ayuda and i[0] not in index:
                index.append(i[0])
                break
    return index

In [10]:
Index_ruleta = Seleccion_ruleta(orden)
print(f"Los cromosomas seleccionados por la ruleta son {Index_ruleta}")

Los cromosomas seleccionados por la ruleta son [8, 2]


# Selección por torneo

La idea principal de este método consiste en realizar la selección en base a comparaciones directas entre individuos. Existen dos versiones de selección mediante torneo:

1. Determinística
2. Probabilística

En la versión determinística se selecciona al azar un número p de individuos (generalmente se escoge $ p=2$). De entre los individuos seleccionados se selecciona el más apto para pasarlo a la siguiente generación.

La versión probabilística únicamente se diferencia en el paso de selección del ganador del torneo. En vez de escoger siempre el mejor se genera un número aleatorio del intervalo [0..1], si es mayor que un parámetro p (fijado para todo el proceso evolutivo) se escoge el individuo más alto y en caso contrario el menos apto. Generalmente p toma valores en el rango $ 0.5 < p \leq 1$.

A continuación, haremos la selección por torneo determinística:

In [12]:
def Seleccion_torneo(FO_Poblacion):
    index = []
    while len(index) < 2:
        flag = True
        while flag == True:
            Padre1 = np.random.randint(len(FO_Poblacion))
            Padre2 = np.random.randint(len(FO_Poblacion))
            
            if Padre1 != Padre2 and Padre1 not in index and Padre2 not in index:
                if FO_Poblacion[Padre1] < FO_Poblacion[Padre2]:
                    index.append(Padre2)
                else:
                    index.append(Padre1)
                        
                flag = False
                
    return index

In [13]:
Index_torneo = Seleccion_torneo(FO_Poblacion)
print(f"Los cromosomas seleccionados por torneo son {Index_torneo}")

Los cromosomas seleccionados por torneo son [8, 9]


# Recombinación o crosover

Una vez seleccionados los individuos, éstos son recombinados para producir la descendencia que se insertará en la siguiente generación. La idea principal del cruce se basa en que, si se toman dos individuos correctamente adaptados al medio y se obtiene una descendencia que comparta genes de ambos, existe la posibilidad de que los genes heredados sean precisamente los causantes de la bondad de los padres. Al compartir las características buenas de dos individuos, la descendencia, o al menos parte de ella, debería tener una bondad mayor que cada uno de los padres por separado. Si el cruce no agrupa las mejores características en uno de los hijos y la descendencia tiene un peor ajuste que los padres no significa que se esté dando un paso atrás.

A continuación se realizará **el Cruce de un solo punto**.

Es la más sencilla de las técnicas de cruce. Una vez seleccionados dos individuos se cortan sus cromosomas por un punto seleccionado aleatoriamente para generar dos segmentos diferenciados en cada uno de ellos: la cabeza y la cola. Se intercambian las colas entre los dos individuos para generar los nuevos descendientes. 

<img src="https://i.ibb.co/LgMqBVK/Cruce1-Punto.png" alt="Cruce1-Punto" border="0">

In [14]:
def Cruce_1_punto(index, n):
    Punto_quiebre = np.random.randint(1, n-1)
    
    Hijo_1 = list(Poblacion[index[0]][0:Punto_quiebre])
    Hijo_1.extend(Poblacion[index[1]][Punto_quiebre:])
    
    Hijo_2 = list(Poblacion[index[1]][0:Punto_quiebre])
    Hijo_2.extend(Poblacion[index[0]][Punto_quiebre:])
    
    return Hijo_1, Hijo_2

In [17]:
print("Padres")
for i in Index_ruleta:
    print(Poblacion[i])

Hijo_1, Hijo_2 = Cruce_1_punto(Index_ruleta,n)

print("Hijos")
print(Hijo_1)
print(Hijo_2)

Padres
[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
Hijos
[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
[0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0]


# Reparación 

Cuando realizamos el procedimiento de cruce puede que el peso de la mochila haya sido sobrepasado en las soluciones de los hijos. Para ello se crea una función de verificación y reparación. En caso de que se haya excedido la capacidad se sacan items de la mochila aleatoriamente hasta recuperar factibilidad. 

In [18]:
def verifico_y_reparo(solucion):
    index_one=[]
    peso_sol = 0
    FO_sol = 0

    for i in range(len(solucion)):
        if solucion[i] == 1: 
            index_one.append(i)
            peso_sol+=w_i[i]
            FO_sol += v_i[i]
    
    if peso_sol <= W:
        return solucion, peso_sol, FO_sol
    else:
        while peso_sol > W:
            index_out = np.random.randint(len(index_one))
            solucion[index_one[index_out]]=0
            peso_sol-=w_i[index_one[index_out]]
            FO_sol-=v_i[index_one[index_out]]
            
            index_one.remove(index_one[index_out])
            
        return solucion, peso_sol, FO_sol

In [19]:
print(Hijo_1)
Hijo_1, Peso_H1, FO_H1 = verifico_y_reparo(Hijo_1)     
print(F"{Hijo_1}, {Peso_H1}, {FO_H1}")

[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1], 1102, 1620


# Mutación

La mutación de un individuo provoca que alguno de sus genes, generalmente uno sólo, varíe su valor de forma aleatoria. En este caso vamos a seleccionar un ítem no empacado de forma aleatoria. En caso de sobrepasar la capacidad de la mochila se utiliza la función de reparación. 

In [20]:
def mutacion(solucion, peso, FO):
    index_cero=[]
    for i in range(len(solucion)):
        if solucion[i] == 0: 
            index_cero.append(i)
    index_in = np.random.randint(len(index_cero))
    solucion[index_cero[index_in]]=1
    peso+=w_i[index_cero[index_in]]
    FO+=v_i[index_cero[index_in]]
    
    if peso > W:
        solucion, peso, FO = verifico_y_reparo(solucion)
        
    return solucion, peso, FO

In [21]:
print(F"{Hijo_1}, {Peso_H1}, {FO_H1}")
Hijo_1, Peso_H1, FO_H1 = mutacion(Hijo_1, Peso_H1, FO_H1)
print(F"{Hijo_1}, {Peso_H1}, {FO_H1}")

[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1], 1102, 1620
[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1], 1042, 1458


# Algoritmo genético

A continuación se presentan el código general del algoritmo genético simple con las función implementadas. 

In [30]:
Ciclos_generacionales = 15 
p_muta = 0.3 
Size = 500

Tiempo_inicio = time.time()

Poblacion, FO_Poblacion, Peso_Poblacion, Incumbente, FO_Incumbente, Peso_Incumbente = Crea_Poblacion(Size, n) #Población inicial

for i in range(Ciclos_generacionales):
    New_Poblacion = []
    New_FO_Poblacion = []
    New_Peso_Poblacion = []

    Total = sum(FO_Poblacion)
    FO_Poblacion_porc = {i : FO_Poblacion[i]/Total for i in range(len(FO_Poblacion))}
    orden = sorted(FO_Poblacion_porc.items(), key=operator.itemgetter(1), reverse=True) 
    
    for j in range(int(Size/2)):
        
        if np.random.rand() <= 0.5:
            Index_ruleta = Seleccion_ruleta(orden)
            Hijo_1, Hijo_2 = Cruce_1_punto(Index_ruleta,n)
        else:
            Index_torneo = Seleccion_torneo(FO_Poblacion)
            Hijo_1, Hijo_2 = Cruce_1_punto(Index_torneo,n)
        
        Hijo_1, Peso_H1, FO_H1 = verifico_y_reparo(Hijo_1)   
        Hijo_2, Peso_H2, FO_H2 = verifico_y_reparo(Hijo_2)   
        
        
        FO_Incumbente, Incumbente, Peso_Incumbente = actualizo_incumbente(FO_Incumbente, Incumbente, Peso_Incumbente, FO_H1, Hijo_1, Peso_H1)
        FO_Incumbente, Incumbente, Peso_Incumbente = actualizo_incumbente(FO_Incumbente, Incumbente, Peso_Incumbente, FO_H2, Hijo_2, Peso_H2)
        
        if np.random.rand() <= p_muta:
            Hijo_1, Peso_H1, FO_H1 = mutacion(Hijo_1, Peso_H1, FO_H1)
            
        if np.random.rand() <= p_muta:
            Hijo_2, Peso_H2, FO_H2 = mutacion(Hijo_2, Peso_H2, FO_H2)
            
        FO_Incumbente, Incumbente, Peso_Incumbente = actualizo_incumbente(FO_Incumbente, Incumbente, Peso_Incumbente, FO_H1, Hijo_1, Peso_H1)
        FO_Incumbente, Incumbente, Peso_Incumbente = actualizo_incumbente(FO_Incumbente, Incumbente, Peso_Incumbente, FO_H2, Hijo_2, Peso_H2)
            
        New_Poblacion.append(Hijo_1)
        New_FO_Poblacion.append(FO_H1)
        Peso_Poblacion.append(Peso_H1)
        
        New_Poblacion.append(Hijo_2)
        New_FO_Poblacion.append(FO_H2)
        Peso_Poblacion.append(Peso_H2)
    
    Poblacion = New_Poblacion.copy()
    FO_Poblacion = New_FO_Poblacion.copy()
    
Tiempo_fin =time.time()-Tiempo_inicio

        
print(f"El beneficio de la mejor solución encontrada es {FO_Incumbente}")
print(f"La mejor solución encontrada es {Incumbente}")
print(f"El peso de la mochila es {Peso_Incumbente}")
print(f"Tiempo de cómputo del algoritmo genético es {round(Tiempo_fin,5)} segundos")

El beneficio de la mejor solución encontrada es 2152
La mejor solución encontrada es [0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1]
El peso de la mochila es 1115
Tiempo de cómputo del algoritmo genético es 0.14404 segundos


En el siguiente código se encuentra el algoritmo constructivo

# Implementación modelo matemático

A continuación se presentan la implementación del problema de mochila 0-1 en Pulp

In [31]:
#!pip install PuLP
import pulp as lp

solucion_MIP = np.zeros(n)

Tiempo_inicio = time.time()
I = range(n)
prob = lp.LpProblem("Knapsack",lp.LpMaximize)

x=lp.LpVariable.dicts("x_var", [i for i in I], lowBound=0,upBound=1,cat="Integer")
prob += lp.lpSum(w_i[i]*x[i] for i in I) <= W, "Capacidad"
prob += lp.lpSum(v_i[i]*x[i] for i in I), "OF"
prob.solve()
Tiempo_fin =time.time()-Tiempo_inicio
    
print("Beneficio total = "+str(lp.value(prob.objective))+"$")
print(f"Tiempo de cómputo del modelo matemático {round(Tiempo_fin,5)} segundos")
FO_MIP = lp.value(prob.objective)
for i in I:
    if x[(i)].varValue > 0:
        solucion_MIP[i] = 1
        #print(f'Paquete {i}: ' + str(x[(i)].varValue))
print(solucion_MIP)

Beneficio total = 2152.0$
Tiempo de cómputo del modelo matemático 0.08548 segundos
[0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1.]
